### ADD Brand List of Shopinng Mall

In [1]:
import sys

sys.path.append("../../")

In [4]:
import json
from typing import List, Dict, Any
from playwright.async_api import Page
from components.dev.shop.shop_product_card_list.schema import ListConfig, ListScrapData
from bs4 import BeautifulSoup, Tag
from components.dev.shop.shop_list.parent_class import PwShopList

class PwAfewStoreList(PwShopList):
    def __name__(self) -> str:
        return "a_few_store"

    def config(self) -> ListConfig:
        return ListConfig(
            scroll_on=True,
            reverse_not_found_result=True,
            page_reload_after_cookies=False,
            not_found_xpath='//div[contains(@class,"findify-components-search--lazy-results")]',
            wait_until_load=10000,
        )

    async def extract_card_html(self, page: Page) -> List[Tag] | None:
        product_cards = await page.query_selector(
            '//div[contains(@class,"findify-components-search--lazy-results")]'
        )
        if product_cards:
            cards = await product_cards.inner_html()
            cards = BeautifulSoup(cards, "html.parser")
            cards = cards.find_all({"attrs": {"class": "product-card"}})
            assert cards, "load_product_card : No product cards found"
            return cards
        else:
            return None

    def extract_info(self, card: Tag) -> ListScrapData:
        price = card.find(class_="price").text  # type: ignore
        id = card["id"].replace("findify_", "")  # type: ignore

        return ListScrapData(
            shop_product_name=card.img["data-src"].split("/")[-1],  # type: ignore
            shop_product_img_url=card.img["src"],  # type: ignore
            product_url=card["href"],  # type: ignore
            product_id=id,
            price=price,
        )

    async def get_next_page(self, page: Page, page_num: int) -> bool:
        xpath = "//button[contains(@class,'findify-components--button btn btn-outline-dark')]"
        button = await page.query_selector(xpath)

        if not button:
            return False

        await button.click()
        return True

In [5]:
from components.dev.shop.shop_product_card_list.list_scraper import ShopListScrapMachine
from components.dev.utils.browser_controller import PwBrowserController
browser_controller = await PwBrowserController().create()
page_controller = await browser_controller.create_page_controller()
module = PwAfewStoreList()

scraper = ShopListScrapMachine(page_controller,module)

FileNotFoundError: [Errno 2] No such file or directory: 'components/dev/cookie/cookies.json'

In [2]:
from typing import List, Dict, Any, Callable

import asyncio 
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import expect, async_playwright, Page, TimeoutError as PlaywrightTimeoutError 



pw = await async_playwright().start()
browser = await pw.firefox.launch(headless=False)
page = await browser.new_page()


page.set_default_timeout(15000)

async def _load_page( page: Page, url):
    """페이지 로드"""
    await asyncio.sleep(1)
    await page.goto(url)
    await page.wait_for_load_state(state="networkidle")
    await page.wait_for_timeout(1000)
    return page

async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)

FileNotFoundError: [Errno 2] No such file or directory: '../../router/dev/kream/cookie/cookies.json'

### shop

In [4]:
url = "https://www.sevenstore.com/accessories/hats-gloves/arcteryx-blue-tetra-bird-head-toque/"

await page.goto(url,wait_until='domcontentloaded',timeout=10000)

async def get_size_info(page: Page) -> List[Dict[str, Any]]:
        locator = page.locator(".product-sizes-title")
        await expect(locator).to_have_text("Sizes", timeout=10000)
        # await expect(page.get_by_text("Sizes")).to_be_visible(timeout=10000)

        size_query = await page.query_selector_all(
            '//div[contains(@class, "size-wrapper")]',
        )

        size_list = [await s.inner_text() for s in size_query]

        if not size_list:
            return [{"shop_product_size": "-", "kor_product_size": "-"}]

        l = []
        for s in size_list:
            kor_size = s
            try:
                if float(s) < 15:
                    kor_size = "UK " + s
            except:
                pass
            l.append({"shop_product_size": s, "kor_product_size": kor_size})

        return l

async def get_product_id(page) -> str:
    product_id_text = await page.query_selector(
        '//meta[contains(@name, "description")]',
    )

    try:
        product_id_text = await product_id_text.get_attribute("content")
        product_id = product_id_text.split(":")[1].replace(" ", "")
    except:
        product_id = "-"

    return product_id.upper()


print(await get_size_info(page))

await get_product_id(page)


None
[{'shop_product_size': 'ONE SIZE', 'kor_product_size': 'ONE SIZE'}]


'X000006756020416'

In [ ]:
size_list

In [ ]:
product_id_text = await page.query_selector('//*[@id="detailsPanel"]/div')
product_id_text = await product_id_text.inner_text()

In [ ]:
text = product_id_text.lower()
if "product code" in text :
    product_id = text.split("product code")[1].strip()
else:
    product_id = '-'

product_id.upper()

### kream

In [ ]:
await _load_page(page, "https://kream.co.kr/login")

In [ ]:
async def save_cookies(page):
    # Get cookies from the current page
    cookies = await page.context.cookies()

    # Save cookies to a file or database
    # In this example, cookies are saved to a file named 'cookies.json'
    with open('../../router/dev/kream/cookie/cookies.json', 'w') as file:
        file.write(str(cookies))

await save_cookies(page)

In [ ]:
async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)
await page.goto("https://kream.co.kr")

In [ ]:
brand = await page.query_selector(".wish_count_num")
brand = await brand.inner_text()
int(brand.replace(",",""))

In [ ]:
x = await page.query_selector_all(".body_list")
volumes = [BeautifulSoup(await i.inner_html()) for i in x]

In [ ]:
# def _extract_volume_data_from(body_list: List) -> List[List]:
#     """
#     크롤링 결과 리스트에서 거래량 추출

#     Args:
#         body_list (List): 크롤링 결과 리스트

#     Returns:
#         List: 크롤링 결과 리스트로 반환
#     """

#     def preprocess_items(tags) -> List:
#         l = []
#         for tag in tags.find_all(class_="list_txt"):
#             l.append(tag.text.lstrip().rstrip())

#         if "빠른" in l[1]:
#             l = l[:1] + l[1].split(" ")[:1] + [True] + l[2:3].replace("빠른배송", "").strip()
#         else:
#             l = l[:2] + [False] + l[2:3].replace("빠른배송", "").strip()
#         return l

#     return [preprocess_items(tags) for tags in body_list]

# body_list = _extract_volume_data_from(volumes)

In [ ]:
x = await page.query_selector_all(".body_list")
volumes = [BeautifulSoup(await i.inner_html()) for i in x]


def _extract_volume_data_from(body_list: List) -> List[List]:
    """
    크롤링 결과 리스트에서 거래량 추출

    Args:
        body_list (List): 크롤링 결과 리스트

    Returns:
        List: 크롤링 결과 리스트로 반환
    """

    def preprocess_items(tags) -> List:
        x = []
        for tag in tags.find_all(class_="list_txt"):
            x.append(tag.text.strip())
        if "빠"in x[1] :
            return [x[0], x[1], True, x[2].replace('빠른배송','').strip()]
        else :
            return [x[0], x[1], False, x[2].replace('빠른배송','').strip()]

    return [preprocess_items(tags) for tags in body_list]

body_list = _extract_volume_data_from(volumes)
body_list